<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/28258187/" target="__blank">Vandervore, et al. (2017) Bi-allelic variants in COL3A1 encoding the ligand to GPR56 are associated with cobblestone-like cortical malformation, white matter changes and cerebellar cysts</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>Please see the <a href="https://monarch-initiative.github.io/pyphetools/" target="__blank">phephetools documentation</a> on the software.</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
from pyphetools.creation import *
from pyphetools.creation.simple_column_mapper import try_mapping_columns, get_separate_hpos_from_df
# last tested with pyphetools version 0.4.5

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155")
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_28258187.xlsx')

In [4]:
df.head()

,Clinical features,Patient 1 (11.3 this manuscript),Patient 2 (11.4 this manuscript),"Patient 3 (Plancke et al, 2009)","Patient 4 (Jergensen et al, 2014)"
0,Sex,female,male,female,female
1,Age at examination(years),7,3.5,10,19
2,Mutation in COL3A1,c.145C>G,c.145C>G,c.479dupT,c.1786C>T
3,Second mutation in COL3A1,c.145C>G,c.145C>G,c.479dupT,c.3851G>A
4,Villefranche criteria,NaN,NaN,NaN,NaN


<h1>Converting to row-based format</h1>
<p>To use pyphetools, we need to have the individuals represented as rows (one row per individual) and have the items of interest be encoded as column names. The required transformations for doing this may be different for different input data, but often we will want to transpose the table (using the pandas <tt>transpose</tt> function) and set the column names of the new table to the zero-th row. After this, we drop the zero-th row (otherwise, it will be interpreted as an individual by the pyphetools code).</p>
<p>After this step is completed, the remaining steps to create phenopackets are the same as in the 
    <a href="http://localhost:8888/notebooks/notebooks/Create%20phenopackets%20from%20tabular%20data%20with%20individuals%20in%20rows.ipynb" target="__blank">row-based notebook</a>.</p>
    
Furthermore, for this specific case, there is a Count features row that we want dropped, so we filter out any row that does not have Patient in the first column.

In [5]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft.head()

Clinical features,Sex,Age at examination(years),Mutation in COL3A1,Second mutation in COL3A1,Villefranche criteria,Major features,Minor features,Additional features,Congenital anomalies,Neurological examination,...,Corpus callosum,Hippocampus,Cerebellum,Cortex,White matter_5,Vermis,Post fossa,Pituitary,Arachnoid cysts,Vessels
Patient 1 (11.3 this manuscript),female,7,c.145C>G,c.145C>G,NaN,-,-,-,-,"Global developmental delay, walks without support, uses a few words",...,"Present, elongated and mildly thickened",Normal,NaN,"Dysplastic cerebellar cortex, multiple cortical cysts superior>inferior","No hypoplasia, multifocal lesions in cerebellar white matter",Vermis hypoplasia cysts,Mega cisterna magna,Normal,NaN,Intracranial segment of the A carotis interna is normal
Patient 2 (11.4 this manuscript),male,3.5,c.145C>G,c.145C>G,NaN,-,-,-,-,"Global developmental delay. sits independently, no words",...,Present elongated and mildly thickened,Normal,NaN,"Dysplastic cerebellar cortex, multiple cortical cysts superior>inferior",No hypoplasia multifocal lesions in cerebellar white matter,Vermis hypoplasia cysts,Mega cisterna magna,Normal,Bilateral temporal pole arachnoidal cysts,Intracranial segment of the A carotis interna is normal
"Patient 3 (Plancke et al, 2009)",female,10,c.479dupT,c.479dupT,NaN,"Easy bruising thin translucent skin, intestinal/arterial tissue fragility. characteristic facial appearance","Early-onset varicose veins, small joint hypermobility","Pulmonary valve stenosis, pronounced atrophic scars, multiple gingival recessions",Talipes equinovarus,"Delayed motor milestones, normal language development",...,"Present, elongated",Normal,NaN,Cortical cysts,Normal,Normal,Normal,Normal,Bilateral temporal pole arachnoidal cysts,NaN
"Patient 4 (Jergensen et al, 2014)",female,19,c.1786C>T,c.3851G>A,NaN,"Easy bruising, thin translucent skin, arterial dissections, characteristic facial appearance","Small joint hypermobility, tendon rupture, first-degree relative with vascular Ehlers- Danlos syndrome","Long, slender fingers",NaN,Normal development and neurological examination,...,Normal,Normal,NaN,Few cortical cysts superior cerebellar lobe,Normal,Mild atrophy,Mega cistema magna,Normal,NaN,"Dilatation left A carotis interna, stenosis right A carotis interna"


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that:

In [15]:
import re
dft.columns = dft.columns.str.strip()
dft = dft.dropna(axis=1, how='all')
dft['patient_id'] = dft.index
# simplify the name of the id column to remove e.g.,(11.3 this manuscript) f
dft['patient_id'] = dft['patient_id'].transform(lambda x: re.sub(r"\(.*?\)", "", x) )
dft.head(2)

Clinical features,Sex,Age at examination(years),Mutation in COL3A1,Second mutation in COL3A1,Major features,Minor features,Additional features,Congenital anomalies,Neurological examination,Head circumference,...,Corpus callosum,Hippocampus,Cortex,White matter_5,Vermis,Post fossa,Pituitary,Arachnoid cysts,Vessels,patient_id
Patient 1 (11.3 this manuscript),female,7,c.145C>G,c.145C>G,-,-,-,-,"Global developmental delay, walks without support, uses a few words",90th centile,...,"Present, elongated and mildly thickened",Normal,"Dysplastic cerebellar cortex, multiple cortical cysts superior>inferior","No hypoplasia, multifocal lesions in cerebellar white matter",Vermis hypoplasia cysts,Mega cisterna magna,Normal,NaN,Intracranial segment of the A carotis interna is normal,Patient 1
Patient 2 (11.4 this manuscript),male,3.5,c.145C>G,c.145C>G,-,-,-,-,"Global developmental delay. sits independently, no words",>97th centile,...,Present elongated and mildly thickened,Normal,"Dysplastic cerebellar cortex, multiple cortical cysts superior>inferior",No hypoplasia multifocal lesions in cerebellar white matter,Vermis hypoplasia cysts,Mega cisterna magna,Normal,Bilateral temporal pole arachnoidal cysts,Intracranial segment of the A carotis interna is normal,Patient 2


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [7]:
hpo_cr = parser.get_hpo_concept_recognizer()
column_mapper_d = defaultdict(ColumnMapper)

Lets try to get code autoformatted so that we can easily copy-paste and change it.

In [9]:
output = OptionColumnMapper.autoformat(df=dft, concept_recognizer=hpo_cr)
print(output)

age_at_examination(years)_d = {'3.5': 'PLACEHOLDER',
 '10': 'PLACEHOLDER',
 '19': 'PLACEHOLDER'}
age_at_examination(years)Mapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=age_at_examination(years)_d)
age_at_examination(years)Mapper.preview_column(df['Age at examination(years)']))
column_mapper_d['Age at examination(years)'] = age_at_examination(years)Mapper

major_features_d = {'Easy bruising thin translucent skin': 'Dermal translucency',
 'intestinal/arterial tissue fragility. characteristic facial appearance': 'PLACEHOLDER',
 'Easy bruising': 'Bruising susceptibility',
 'thin translucent skin': 'Dermal translucency',
 'arterial dissections': 'Arterial dissection',
 'characteristic facial appearance': 'PLACEHOLDER'}
major_featuresMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=major_features_d)
major_featuresMapper.preview_column(df['Major features']))
column_mapper_d['Major features'] = major_featuresMapper

minor_features_d = {'Early-onset varicose vei

In [10]:
                                         
major_features = {'Easy bruising thin translucent skin': 'Bruising susceptibility',
 'arterial tissue fragility': 'Abnormal arterial physiology',
 'Easy bruising': 'Bruising susceptibility',
 'thin translucent skin': 'Dermal translucency',
 'arterial dissections': 'Arterial dissection',}
major_featuresMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=major_features)
#print(major_featuresMapper.preview_column(dft['Major features']))
column_mapper_d['Major features'] = major_featuresMapper

cortex = {'Dysplastic cerebellar cortex': 'Abnormal cerebellar cortex morphology'}
cortexMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=cortex)
#print(cortexMapper.preview_column(dft['Cortex']))
column_mapper_d['Cortex'] = cortexMapper

minor_features = {'Early-onset varicose veins': 'Varicose veins',
 'small joint hypermobility': 'Joint hypermobility',
 'tendon rupture': 'Tendon rupture'}
minor_featuresMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=minor_features)
#print(minor_featuresMapper.preview_column(dft['Minor features']))
column_mapper_d['Minor features'] = minor_featuresMapper

additional_features = {'Pulmonary valve stenosis': 'Pulmonic stenosis',
 'pronounced atrophic scars': 'Atrophic scars',
 'multiple gingival recessions': 'Gingival recession',
 'slender fingers': 'Slender finger'}
additional_featuresMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=additional_features)
#print(major_featuresMapper.preview_column(dft['Additional features']))
column_mapper_d['Additional features'] = additional_featuresMapper


congenital_anomalies = {'Talipes equinovarus': 'Talipes equinovarus'}
congenital_anomaliesMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=congenital_anomalies)
#print(congenital_anomaliesMapper.preview_column(dft['Congenital anomalies']))
column_mapper_d['Congenital anomalies'] = congenital_anomaliesMapper

neurological_examination = {'Global developmental delay': 'Global developmental delay',
 'no words': 'Absent speech',
 'Delayed motor milestones': 'Motor delay'}
neurological_examinationMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=neurological_examination)
#print(neurological_examinationMapper.preview_column(dft['Neurological examination']))
column_mapper_d['Neurological examination'] = neurological_examinationMapper

head_circumference = {'>97th centile': 'Macrocephaly'}
head_circumferenceMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=head_circumference)
#print(head_circumferenceMapper.preview_column(dft['Head circumference']))
column_mapper_d['Head circumference'] = head_circumferenceMapper

epilepsy_onset = {'Spasms/5 years': 'Seizure',
 'Spasms/26 months': 'Seizure',
 'Absence seizures/unknown': 'Typical absence seizure'}
epilepsy_onsetMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=epilepsy_onset)
#print(epilepsy_onsetMapper.preview_column(dft['Epilepsy/onset']))
column_mapper_d['Epilepsy/onset'] = epilepsy_onsetMapper


gyral_pattern = {'Diffuse thickened cobblestone cortex with relative sparing of the temporal lobes': 'Dysgyria with thickened cortex',
 'Diffuse thickened cobblestone cortex with relative sparing of the temporal poles': 'Dysgyria with thickened cortex',
 'Frontal cobblestone cortex': 'Dysgyria with thickened cortex',
 'parietal polymicrogyria': 'Polymicrogyria',
 'Bilateral frontal polymicrogyria including cingulate gyri': 'Polymicrogyria'}
gyral_patternMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=gyral_pattern)
#print(gyral_patternMapper.preview_column(dft['Gyral pattern']))
column_mapper_d['Gyral pattern'] = gyral_patternMapper


white_matter = {'Globale reduction of white matter': 'Hypointensity of cerebral white matter on MRI',
 'Diffuse hypomyelination': 'Cerebral hypomyelination'}
white_matterMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=white_matter)
#print(white_matterMapper.preview_column(dft['White matter']))
column_mapper_d['White matter'] = white_matterMapper

white_matter_2 = {'Prominent perivascular spaces': 'Dilation of Virchow-Robin spaces',
 'Prominent perivascular spaces bilateral frontal': 'Dilation of Virchow-Robin spaces'}
white_matter_2Mapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=white_matter_2)
#print(white_matter_2Mapper .preview_column(dft['White matter_2']))
column_mapper_d['White matter_2'] = white_matter_2Mapper

white_matter_3 = {'Frontal nodular heterotopia (beads)': 'Gray matter heterotopia',
 'perisylvian and occipital band heterotopia': 'Gray matter heterotopia'}
white_matter_3Mapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=white_matter_3)
#print(white_matter_3Mapper.preview_column(dft['White matter_3']))
column_mapper_d['White matter_3'] = white_matter_3Mapper


lateral_ventricles = {'Ventriculomegaly': 'Lateral ventricle dilatation',
 'Mild enlargement': 'Lateral ventricle dilatation'}
lateral_ventriclesMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=lateral_ventricles)
#print(lateral_ventriclesMapper.preview_column(dft['Lateral ventricles']))
column_mapper_d['Lateral ventricles'] = lateral_ventriclesMapper

third_ventricle = {'Ventriculomegaly': 'Dilated third ventricle',
 'Mild enlargement': 'Dilated third ventricle'}
third_ventricleMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=third_ventricle)
#print(third_ventricleMapper.preview_column(dft['Third ventricle']))
column_mapper_d['Third ventricle'] = third_ventricleMapper

brainstem = {'Hypoplastic': 'Abnormal brainstem morphology',
 'Mildly hypoplastic': 'Abnormal brainstem morphology',
 'Hypoplasia of the pons': 'Hypoplasia of the pons'}
brainstemMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=brainstem)
#print(brainstemMapper.preview_column(dft['Brainstem']))
column_mapper_d['Brainstem'] = brainstemMapper

basal_ganglia = {'Thalamus normal putamen/globus pallidus small': 'Abnormal globus pallidus morphology',
 'accentuated Virchow- Robin spaces': 'Dilation of Virchow-Robin spaces'}
basal_gangliaMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=basal_ganglia)
#print(basal_gangliaMapper.preview_column(dft['Basal ganglia']))
column_mapper_d['Basal ganglia'] = basal_gangliaMapper

corpus_callosum = {'elongated and mildly thickened': 'Abnormal length of corpus callosum',
 'Present elongated and mildly thickened': 'Abnormal length of corpus callosum',
 'elongated': 'Abnormal length of corpus callosum'}
corpus_callosumMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=corpus_callosum)
#print(corpus_callosumMapper.preview_column(dft['Corpus callosum']))
column_mapper_d['Corpus callosum'] = corpus_callosumMapper

vermis = {'Vermis hypoplasia cysts': 'Cerebellar vermis hypoplasia',
 'Mild atrophy': 'Cerebellar vermis hypoplasia'}
vermisMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=vermis)
#print(vermisMapper.preview_column(dft['Vermis']))
column_mapper_d['Vermis'] = vermisMapper

post_fossa = {'Mega cisterna magna': 'Enlarged cisterna magna',
 'Mega cistema magna': 'Enlarged cisterna magna'}
post_fossaMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=post_fossa)
#print(post_fossaMapper.preview_column(dft['Post fossa']))
column_mapper_d['Post fossa'] = post_fossaMapper

vessels = { 'Dilatation left A carotis interna': 'Carotid artery dilatation',
 'stenosis right A carotis interna': 'Carotid artery stenosis'}
vesselsMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=vessels)
#print(vesselsMapper.preview_column(dft['Vessels']))
column_mapper_d['Vessels'] = vesselsMapper

arachnoid_cysts = {'Bilateral temporal pole arachnoidal cysts': 'Arachnoid cyst'}
arachnoid_cystsMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=arachnoid_cysts)
#print(arachnoid_cystsMapper.preview_column(dft['Arachnoid cysts']))
column_mapper_d['Arachnoid cysts'] = arachnoid_cystsMapper


<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [11]:
genome = 'hg38'
default_genotype = 'homozygous'
transcript='NM_000090.3'
varMapper = VariantColumnMapper(assembly=genome,column_name='Mutation in COL3A1', 
                                transcript=transcript, default_genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [12]:
ageMapper = AgeColumnMapper.by_year('Age at examination(years)')
ageMapper.preview_column(dft['Age at examination(years)'])

,original column contents,age
0,7,P7Y
1,3.5,P3Y6M
2,10,P10Y
3,19,P19Y


In [18]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='Sex')
sexMapper.preview_column(dft['Sex'])

,original column contents,sex
0,female,FEMALE
1,male,MALE
2,female,FEMALE
3,female,FEMALE


In [19]:
pmid = "PMID:28258187"
encoder = CohortEncoder(df=dft, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='618343', label='Polymicrogyria with or without vascular-type EDS')

In [20]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.145C>G/NM_000090.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.145C>G/NM_000090.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.479dupT/NM_000090.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.1786C>T/NM_000090.3?content-type=application%2Fjson


In [22]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "Patient 1 (11.3 this manuscript)",
  "subject": {
    "id": "Patient 1 (11.3 this manuscript)",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P7Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0031422",
        "label": "Abnormal cerebellar cortex morphology"
      }
    },
    {
      "type": {
        "id": "HP:0001263",
        "label": "Global developmental delay"
      }
    },
    {
      "type": {
        "id": "HP:0032400",
        "label": "Dysgyria with thickened cortex"
      }
    },
    {
      "type": {
        "id": "HP:0007103",
        "label": "Hypointensity of cerebral white matter on MRI"
      }
    },
    {
      "type": {
        "id": "HP:0012520",
        "label": "Dilation of Virchow-Robin spaces"
      }
    },
    {
      "type": {
        "id": "HP:0006956",
        "label": "Lateral ventricle dilatation"
      }
    },
    {
      "type": {
        "id": "HP:00

In [23]:
output_directory = "../../phenopackets/COL3A1/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.145C>G/NM_000090.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.145C>G/NM_000090.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.479dupT/NM_000090.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000090.3%3Ac.1786C>T/NM_000090.3?content-type=application%2Fjson
Wrote 4 phenopackets to ../../phenopackets/COL3A1/
